In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from time import time


from keras.datasets import cifar10
import random
import cv2
import numpy as np
from keras.utils import to_categorical

def resize_data(data):
    data_upscaled = np.zeros((data.shape[0], 240, 240, 3))
    for i, img in enumerate(data):
        large_img = cv2.resize(img, dsize=(240, 240), interpolation=cv2.INTER_CUBIC)
        data_upscaled[i] = large_img

    return data_upscaled

c:\users\jesper\appdata\local\conda\conda\envs\tensorflowv3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load the data
# read data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# limit the amount of the data
Nmax = 200
C = 10
# train data
ind_train = random.sample(list(range(x_train.shape[0])), Nmax)
x_train = x_train[ind_train]
y_train = y_train[ind_train]

# test data
ind_test = random.sample(list(range(x_test.shape[0])), Nmax)
x_test = x_test[ind_test]
y_test = y_test[ind_test]

In [3]:
# Preprare the data
# resize train and  test data
x_train_resized = resize_data(x_train)
x_test_resized = resize_data(x_test)

# make explained variable hot-encoded
y_train_hot_encoded = to_categorical(y_train)
y_test_hot_encoded = to_categorical(y_test)

In [4]:
# Configure model
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(C, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [5]:
# Fit the model for the top layers
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit(x_train_resized, y_train_hot_encoded)

Epoch 1/1
200/200 [==============================] - 15s 73ms/step - loss: 7.3739


In [6]:
# Fit the model for the lower layers
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

tbCallBack = keras.callbacks.TensorBoard(log_dir="../../LocalContent/Logs/KerasRetrainInceptionv3_3/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)


# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit(x_train_resized, y_train_hot_encoded, epochs=5,callbacks=[tbCallBack])

Instructions for updating:
Use the retry module or similar alternatives.
Epoch 1/5
200/200 [==============================] - 13s 64ms/step - loss: 8.6185 - acc: 0.1100
Epoch 2/5
200/200 [==============================] - 6s 30ms/step - loss: 6.2778 - acc: 0.1300
Epoch 3/5
200/200 [==============================] - 6s 30ms/step - loss: 4.6617 - acc: 0.2850
Epoch 4/5
200/200 [==============================] - 6s 30ms/step - loss: 4.1951 - acc: 0.3750
Epoch 5/5
200/200 [==============================] - 6s 30ms/step - loss: 3.8293 - acc: 0.5050
